In [17]:
# TODO: refine labels using ratings
# also add to data pipeline

In [18]:
%%capture
from pathlib import Path

if Path.cwd().stem == "features":
    %cd ../..
    %load_ext autoreload
    %autoreload 2

In [19]:
import logging
import operator
from functools import reduce

import holoviews as hv
import hvplot.polars  # noqa
import numpy as np
import polars as pl
from icecream import ic
from polars import col

from src.data.database_manager import DatabaseManager
from src.data.quality_checks import check_sample_rate
from src.experiments.measurement.stimulus_generator import StimulusGenerator
from src.features.labels import process_labels
from src.features.resampling import add_time_column, downsample
from src.features.scaling import scale_min_max
from src.features.transforming import map_trials, merge_dfs
from src.log_config import configure_logging
from src.plots.plot_stimulus import plot_stimulus_with_shapes

configure_logging(
    stream_level=logging.DEBUG,
    ignore_libs=["matplotlib", "Comm", "bokeh", "tornado"],
)

pl.Config.set_tbl_rows(12)  # for the 12 trials
hv.output(widget_location="bottom", size=130)

In [20]:
db = DatabaseManager()

In [21]:
with db:
    stimulus = db.get_table("Feature_Stimulus")
    eda = db.get_table("Feature_EDA")
    trials = db.get_table("Trials")

df = merge_dfs([stimulus, trials], on=["trial_id", "participant_id", "trial_number"])


In [22]:
from src.features.labels import add_labels

df = add_labels(eda, trials)
df

trial_id,trial_number,participant_id,rownumber,timestamp,samplenumber,eda_raw,eda_tonic,eda_phasic,stimulus_seed,skin_area,normalized_timestamp,decreasing_intervals,major_decreasing_intervals,increasing_intervals,strictly_increasing_intervals,plateau_intervals,prolonged_minima_intervals
u16,u8,u8,u32,f64,i64,f64,f64,f64,u16,u8,f64,u16,u16,u16,u16,u16,u16
1,1,1,37660,294210.3603,57892,0.743774,0.743503,0.000271,396,1,0.0,0,0,1,0,0,0
1,1,1,37673,294340.0132,57905,0.745042,0.743523,0.001519,396,1,129.6529,0,0,1,0,0,0
1,1,1,37687,294421.1276,57919,0.744819,0.743543,0.001276,396,1,210.7673,0,0,1,0,0,0
1,1,1,37701,294528.84,57933,0.744804,0.743562,0.001242,396,1,318.4797,0,0,1,0,0,0
1,1,1,37713,294616.4458,57945,0.744887,0.743581,0.001305,396,1,406.0855,0,0,1,0,0,0
1,1,1,37726,294737.124,57958,0.744989,0.7436,0.001389,396,1,526.7637,0,0,1,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
332,12,28,355419,2.7766e6,467016,13.441099,13.423482,-0.069582,133,1,179521.2445,1660,0,0,0,0,0
332,12,28,355431,2.7767e6,467028,13.464384,13.423471,-0.051391,133,1,179609.0093,1660,0,0,0,0,0
